In [1]:
BASE_URL = "https://www.otodom.pl"
SEARCH_URL = BASE_URL + "/pl/wyniki/sprzedaz/mieszkanie/malopolskie/krakow/krakow/krakow?viewType=listing&page="
ulr = SEARCH_URL + "1"
print(ulr)

https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/malopolskie/krakow/krakow/krakow?viewType=listing&page=1


In [2]:
import requests
from bs4 import BeautifulSoup

def load_page(url: str) -> BeautifulSoup:
    res = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

def get_links_for_search_page(n: int) -> list[str]:
    url = SEARCH_URL + str(n)
    soup = load_page(url)
    links = [BASE_URL + a['href'] for a in soup.find_all('a', attrs={'data-cy': 'listing-item-link'})]
    return links

In [3]:
links = get_links_for_search_page(1)
links[0]

'https://www.otodom.pl/pl/oferta/4-pokojowe-mieszkanie-65m2-ogrodek-bezposrednio-ID4ulBE'

In [20]:
def extract_data(url: str, soup: BeautifulSoup):
    print(f"Extracting data from {url}")
    slug = url.split('/')[-1] # this is unique identifier which we can use as primary key
    name = soup.find('h1', attrs={'data-cy': 'adPageAdTitle'}).text
    price = soup.find('strong', attrs={'aria-label': 'Cena'}).text
    info = soup.find_all('div', attrs={'class': 'css-1ftqasz'})
    area = info[0].text
    rooms = info[1].text
    print(f"Extracted data: {name}, {price}, {area}, {rooms}")
    
    info_table = soup.find('div', {'class': 'css-t7cajz e15n0fyo1'}).parent
    info_table = [element for element in info_table.children if element.name == 'div']
    cols = ['heating', 'floor', 'rent', 'state', 'market', 'ownership', 'available', 'ad_type', 'extra_info']
    data = {col: info_table[idx].text for col, idx in zip(cols, range(0, 18, 2))}
    print(f"Data from table: {data}")

    data.update({'slug': slug, 'price': price, 'name': name, 'area': area, 'rooms': rooms})
    return data

In [14]:
import pandas as pd

def get_one_search_page(n: int) -> pd.DataFrame:
    links = get_links_for_search_page(n)
    df = pd.DataFrame()
    data_list = []
    for link in links:
        print(f"Checking {link}")
        soup = load_page(link)
        data = extract_data(link, soup)
        data_list.append(data)
    df = pd.concat([df, pd.DataFrame(data_list)], ignore_index=True)
    df.set_index('slug', inplace=True)
    return df

In [18]:
def get_n_pages(n: int, offset:int=0) -> pd.DataFrame:
    df = pd.DataFrame()
    for i in range(offset, n):
        df = pd.concat([df, get_one_search_page(i)], ignore_index=False)
    return df

In [21]:
df = get_n_pages(1)

Checking https://www.otodom.pl/pl/oferta/krakow-krowodrza-wroclawska-52a-kawalerka-ID4uISs
Extracting data from https://www.otodom.pl/pl/oferta/krakow-krowodrza-wroclawska-52a-kawalerka-ID4uISs
Extracted data: Kraków/Krowodrza/Wrocławska 52A/kawalerka, 339 000 zł, 17.01m², 1 pokój


AttributeError: 'NoneType' object has no attribute 'parent'

In [145]:
# Check what we've got - random 5 rows
print(df.shape)
df.sample(5)

(69, 13)


,heating,floor,rent,state,market,ownership,available,ad_type,extra_info,price,name,area,rooms
slug,,,,,,,,,,,,,
ul-powstancow-mieszkanie-3-pok-e-38-ID4tLF7,Ogrzewanie:miejskie,Piętro:parter/9,Czynsz:brak informacji,Stan wykończenia:do wykończenia,Rynek:pierwotny,Forma własności:pełna własność,Dostępne od:2025-06-30,Typ ogłoszeniodawcy:deweloper,Informacje dodatkowe:balkon garaż/miejsce park...,760 158 zł,ul. Powstańców | mieszkanie 3-pok. | E 38,60.33m²,3 pokoje
4-pokojowe-mieszkanie-90m2-2-balkony-ID4ulXQ,Ogrzewanie:brak informacji,Piętro:1/7,Czynsz:brak informacji,Stan wykończenia:do wykończenia,Rynek:pierwotny,Forma własności:pełna własność,Dostępne od:2025-11-15,Typ ogłoszeniodawcy:deweloper,Informacje dodatkowe:balkon garaż/miejsce park...,1 648 653 zł,4-pokojowe mieszkanie 90m2 + 2 balkony,90.4m²,4 pokoje
green-apartments-2-0-petrazyckiego-nowa-inwestycja-ID4ehG4,Ogrzewanie:gazowe,Piętro:1/2,Czynsz:brak informacji,Stan wykończenia:do wykończenia,Rynek:pierwotny,Forma własności:pełna własność,Dostępne od:brak informacji,Typ ogłoszeniodawcy:prywatny,Informacje dodatkowe:balkon garaż/miejsce park...,734 000 zł,Green Apartments 2.0 Petrażyckiego NOWA INWEST...,77.89m²,4 pokoje
4-pokojowe-mieszkanie-146m2-ogrodek-bez-prowizji-ID4uhOs,Ogrzewanie:brak informacji,Piętro:parter/1,Czynsz:brak informacji,Stan wykończenia:do wykończenia,Rynek:pierwotny,Forma własności:pełna własność,Dostępne od:2026-05-15,Typ ogłoszeniodawcy:deweloper,Informacje dodatkowe:taras ogródek garaż/miejs...,Zapytaj o cenę,4-pokojowe mieszkanie 146m2 + ogródek Bez Prow...,146.8m²,4 pokoje
3-pokojowe-mieszkanie-60m2-balkon-ID4uomh,Ogrzewanie:brak informacji,Piętro:8/10,Czynsz:brak informacji,Stan wykończenia:do wykończenia,Rynek:pierwotny,Forma własności:pełna własność,Dostępne od:2025-05-05,Typ ogłoszeniodawcy:deweloper,Informacje dodatkowe:balkon garaż/miejsce park...,861 000 zł,3-pokojowe mieszkanie 60m2 + balkon,60.18m²,3 pokoje


In [148]:
df.to_csv('otodom.csv')